<h1 align=center><font size=8> Assignment 01 ADY201m 

#### Import Libaries

In [1]:
import pandas as pd
import pyodbc as odbc


In [2]:
# pd.set_option('display.max_rows', None)

#### Connect to Database

In [3]:
connection = odbc.connect('Driver={ODBC Driver 18 for SQL Server};\
                          Server=tcp:adyserver.database.windows.net,1433;Database=ADY201m_AS01;\
                          Uid=thang99;Pwd={Matkhau@123};Encrypt=yes;\
                          TrustServerCertificate=no;Connection Timeout=30;')

cursor = connection.cursor()

#### Solving Problems

##### Problem 1
Find the total number of crimes recorded in the CRIME table.

In [4]:
querry_1 = ("SELECT COUNT(*) AS Total_Number_Crimes FROM tblChicagoCrimes")
df_1 = pd.read_sql(querry_1, con=connection)
df_1

C:\Users\admin\AppData\Local\Temp\ipykernel_32152\505866151.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_1 = pd.read_sql(querry_1, con=connection)


,Total_Number_Crimes
0,8068562


##### Problem 2
List community areas with per capita income less than 11000.

In [5]:
querry_2 = ("""
    SELECT Community_Area_Name
    FROM tblCommunity
    JOIN tblChicagoSocioeconomic
    ON tblCommunity.Community_Area_Number = tblChicagoSocioeconomic.Community_Area_Number
    WHERE PER_CAPITA_INCOME < 11000
""")
df_2 = pd.read_sql(querry_2, con=connection)
df_2

C:\Users\admin\AppData\Local\Temp\ipykernel_32152\685738995.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_2 = pd.read_sql(querry_2, con=connection)


,Community_Area_Name
0,WEST GARFIELD PARK
1,SOUTH LAWNDALE
2,FULLER PARK
3,RIVERDALE


##### Problem 3
List all case numbers for crimes involving minors.

In [6]:
query_3 = ("""
    SELECT DISTINCT C.Case_Number 
    FROM tblChicagoCrimes C
    JOIN tblIUCR I ON C.IUCR = I.IUCR
    WHERE I.Description LIKE '%minor%' 
""")
df_3 = pd.read_sql(query_3, con=connection)
df_3

C:\Users\admin\AppData\Local\Temp\ipykernel_32152\1577008993.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_3 = pd.read_sql(query_3, con=connection)


,Case_Number
0,JA122123
1,HK425438
2,HY308502
3,JD169961
4,HH616169
...,...
33644,HP418172
33645,JC406959
33646,JD449862
33647,HJ380514


##### Problem 4
List all kidnapping crimes involving a child? (children are not considered minors for the purposes of crime analysis)

In [7]:
query_4 = """ 
SELECT cc.Case_Number, ca.Location_Description, cc.Arrest
FROM tblChicagoCrimes cc
JOIN tblCrimeAddress ca ON cc.ID = ca.ID
JOIN tblIUCR iucr ON cc.IUCR = iucr.IUCR
WHERE iucr.Primary_Type = 'KIDNAPPING'
AND iucr.Description LIKE '%CHILD%'
"""
df_4 = pd.read_sql(query_4, connection)

df_4

C:\Users\admin\AppData\Local\Temp\ipykernel_32152\3963878566.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_4 = pd.read_sql(query_4, connection)


,Case_Number,Location_Description,Arrest
0,HH326898,STREET,True
1,HH323240,STREET,False
2,HH332570,STREET,False
3,HH330947,STREET,False
4,HH338213,STREET,False
...,...,...,...
2344,JH225739,RESIDENCE,False
2345,JH230751,STREET,False
2346,JH252227,STREET,False
2347,JH263557,RESIDENCE,False


##### Problem 5
What kind of crime were recorded at schools?

In [8]:
query_5 = """
SELECT DISTINCT(iucr.Primary_Type)
FROM tblChicagoCrimes cc
JOIN tblCrimeAddress ca ON cc.ID = ca.ID
JOIN tblIUCR iucr ON cc.IUCR = iucr.IUCR
WHERE ca.Location_Description LIKE '%SCHOOL%' 
"""

df_5 = pd.read_sql(query_5, con=connection)

df_5

C:\Users\admin\AppData\Local\Temp\ipykernel_32152\1860079712.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_5 = pd.read_sql(query_5, con=connection)


,Primary_Type
0,ARSON
1,ASSAULT
2,BATTERY
3,BURGLARY
4,CONCEALED CARRY LICENSE VIOLATION
5,CRIM SEXUAL ASSAULT
6,CRIMINAL DAMAGE
7,CRIMINAL SEXUAL ASSAULT
8,CRIMINAL TRESPASS
9,DECEPTIVE PRACTICE


##### Problem 6
List the average safety score for all types of schools.

In [9]:
query_6 = """
SELECT cps.Elementary_Middle_or_High_School, AVG(sr.Safety_Score) AS Average_Safety_Score
FROM tblChicagoPublicSchools cps
JOIN tblSurvey_Results sr ON cps.School_ID = sr.School_ID
GROUP BY cps.Elementary_Middle_or_High_School
ORDER BY Average_Safety_Score
"""

df_6 = pd.read_sql(query_6, con=connection)

df_6

C:\Users\admin\AppData\Local\Temp\ipykernel_32152\147346347.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_6 = pd.read_sql(query_6, con=connection)


,Elementary_Middle_or_High_School,Average_Safety_Score
0,MS,48.000000
1,ES,49.520384
2,HS,49.623529


##### Problem 7
List 5 community areas with highest % of households below poverty line.

In [10]:
query_7 = ('''
    SELECT TOP 5 C.Community_Area_Name, CS.PERCENT_HOUSEHOLDS_BELOW_POVERTY 
    FROM tblChicagoSocioeconomic CS
    INNER JOIN tblCommunity C ON CS.Community_Area_Number = C.Community_Area_Number
    ORDER BY CS.PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC;
''')


In [11]:
df_7 = pd.read_sql(query_7, con=connection)
df_7

C:\Users\admin\AppData\Local\Temp\ipykernel_32152\3544942599.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_7 = pd.read_sql(query_7, con=connection)


,Community_Area_Name,PERCENT_HOUSEHOLDS_BELOW_POVERTY
0,RIVERDALE,56.5
1,FULLER PARK,51.2
2,ENGLEWOOD,46.6
3,NORTH LAWNDALE,43.1
4,EAST GARFIELD PARK,42.4


##### Problem 8
Which community area(number) is most crime prone?

In [27]:
query_8 = '''
    WITH A AS (
    SELECT  CA.Community_Area_Number, 
        COUNT(CASE WHEN CC.Arrest = 0 THEN 1 END) AS Non_Arrest_Count,
        COUNT(CASE WHEN CC.Arrest = 1 THEN 1 END) AS Arrest_Count
    FROM tblCrimeAddress CA
    INNER JOIN tblChicagoCrimes CC ON CA.ID = CC.ID
    GROUP BY CA.Community_Area_Number)

    SELECT TOP 1 Community_Area_Number,
            (CAST(Non_Arrest_Count AS FLOAT)/CAST((Non_Arrest_Count+Arrest_Count) AS FLOAT) * 100) AS Non_Arrest_Percent
    FROM A
    ORDER BY Non_Arrest_Percent DESC;
'''

In [28]:
df_8 = pd.read_sql(query_8, con=connection)
df_8

C:\Users\admin\AppData\Local\Temp\ipykernel_32152\3790869786.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_8 = pd.read_sql(query_8, con=connection)


,Community_Area_Number,Non_Arrest_Percent
0,12,89.634014


#### Problem 9
Use a sub-query to find the name of the community area with highest hardship index.

In [14]:
query_9 = '''
    SELECT Community_Area_Name FROM tblCommunity C
    WHERE C.Community_Area_Number = (
        SELECT TOP 1 Community_Area_Number FROM tblChicagoSocioeconomic CS
        ORDER BY HARDSHIP_INDEX DESC
        )
'''

In [15]:
df_9 = pd.read_sql(query_9, con=connection)
df_9

C:\Users\admin\AppData\Local\Temp\ipykernel_32152\725692994.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_9 = pd.read_sql(query_9, con=connection)


,Community_Area_Name
0,RIVERDALE


#### Problem 10
Use a sub-query to determine the Community Area Name with most number of crimes?

In [16]:
querry_10 = '''
    SELECT Community_Area_Name FROM tblCommunity
    WHERE Community_Area_Number = (
        SELECT TOP 1 Community_Area_Number FROM tblCrimeAddress
        GROUP BY Community_Area_Number
        ORDER BY COUNT(Community_Area_Number) DESC
        )
'''

In [17]:
df_10 = pd.read_sql(querry_10, con=connection)
df_10

C:\Users\admin\AppData\Local\Temp\ipykernel_32152\751850723.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_10 = pd.read_sql(querry_10, con=connection)


,Community_Area_Name
0,AUSTIN
